In [1]:
import threading

# Создание потоков

In [2]:
def factorial(n: int, thread_num: int) -> int:
    res = 1
    for i in range(1, n + 1):
        res *= i
    
    print('Thread {}: {}\n'.format(thread_num, res))
    return res

thread1 = threading.Thread(target=factorial, args=(5, 1))
thread2 = threading.Thread(target=factorial, args=(100, 2))

thread1.start()
thread2.start()

thread1.join()
thread2.join()

Thread 1: 120

Thread 2: 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000



# ThreadPoolExecutor

Создание поток - дорогая операция. `ThreadPoolExecutor` - это пул поток, методв `submit` назначает заданию какой-то свободный поток (если есть свободные, иначе ставит в очередь). 

`Future` (оно же в определенном варианте `Promise`, `CompletableFuture`) - это некоторая ручка, которая позволяет отслеживать процесс выполнения задания, получать результат и т.п.

In [3]:
from concurrent.futures import Future, ThreadPoolExecutor


def factorial(n: int) -> int:
    res = 1
    for i in range(1, n + 1):
        res *= i
    
    return res


def add(a: int, b: int) -> int:
    return a + b


executor = ThreadPoolExecutor(max_workers=3)

future1 = executor.submit(factorial, 5)
future2 = executor.submit(factorial, 10)
future3 = executor.submit(add, 5, 3)

future3.result(), future1.result(), future2.result()

(8, 120, 3628800)

Есть многопоточный аналог функции `map`

In [4]:
# input list
it = [x for x in range(10000)]

# list comprehension
lst = [x**2 for x in it]
# map
lst = list(map(lambda x: x**2, it))

In [5]:
# "parallel" map computation
it = [x * 10 for x in range(10000)]
for r in executor.map(lambda x: x ** 2, it):
    pass

# Процессы

Потоки в `CPython` неэффективны для вычислительно-сложный операций, так как присутствут `GIL`. В качестве альтернативы есть модуль `multiprocessing`, который создает процессы вместо потоков. Новый процесс имеет своё адресное пространство памяти, потому взаимодействие между процессами некоторым образом осложнено. Более того, метод создание дочерних процессов может различаться в зависимости от операционной системы.

In [6]:
# Загружаем параллельно три ссылки

import time
from multiprocessing import Pool, Process


with Pool() as pool:    
    fut1 = pool.apply_async(factorial, args=(100,))
    fut2 = pool.apply_async(factorial, args=(200,))
    print(fut1.get(), fut2.get())

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000 788657867364790503552363213932185062295135977687173263294742533244359449963403342920304284011984623904177212138919638830257642790242637105061926624952829931113462857270763317237396988943922445621451664240254033291864131227428294853277524242407573903240321257405579568660226031904170324062351700858796178922222789623703897374720000000000000000000000000000000000000000000000000


In [7]:
# Загружаем параллельно три ссылки

import time
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor


with ProcessPoolExecutor(4) as pool:
    fut1 = pool.submit(factorial, 100)
    fut2 = pool.submit(factorial, 200)
    print(fut1.result(), fut2.result())

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000 788657867364790503552363213932185062295135977687173263294742533244359449963403342920304284011984623904177212138919638830257642790242637105061926624952829931113462857270763317237396988943922445621451664240254033291864131227428294853277524242407573903240321257405579568660226031904170324062351700858796178922222789623703897374720000000000000000000000000000000000000000000000000
